In [40]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import datetime

from nltk.stem.snowball import SnowballStemmer
from nltk import word_tokenize

In [41]:
#read the course info
courses=pd.read_csv('../Data/filtered_courses.csv')
#fix string list to make it just a string
courses['teachers']=courses['teachers'].str.replace("[\[,'\]]", '')
courses=courses.fillna('')
print("Number of courses:",len(courses),"\n")
courses.head()

Number of courses: 1212 



,index,additionalInformation,assesmentMethods,availableEnglish,cefrLevel,code,content,courseStatus,courseUnitId,credits,...,organizationId,prerequisities,registration,startDate,substitutes,teacherInCharge,teachers,teachingPeriod,type,workload
0,0,Compulsory attendance in all class sessions an...,100 % assignments (group and individual),True,,20E99904,"The course consists of an applied, real-life p...",Mandatory course in the Master¿s programs of B...,1125574316,6,...,E701,Most Master¿s Programme studies have to be com...,via WebOodi.,2018-09-19,Students can replace this capstone course by p...,Perttu KähäriNina GranqvistPaulina JunniGregor...,Perttu Kähäri Laura Peni Pekka Pälli Iiris Sai...,"Periods I-II Töölö campus, periods IV-V Otanie...",course,Contact teaching :10-15 h (incl. closing semin...
1,1,The minimum number of participants is 20,Learning diaries 50%Take-home exam 50%,True,,21C00150,This introductory course gives a basic underst...,Degree Elective,1130843834,3,...,E706,,Via WebOodi,2019-02-27,,"DSc Christa Uusi-Rauva, Professor Ingmar Björkman",Alice Wickström Ingmar Björkman,"2018-2019; IV, Otaniemi Campus 2019-2020: no t...",course,Lectures: 33 hoursLearning diaries: 24 hoursTa...
2,2,Max. 100 students. Priority for management stu...,Final exam: 40%Assignments: 30%Learning diary:...,True,,21C00350,"Throughout this course, we will be covering di...",Bachelor: Management HR specialization area Co...,1125857456,6,...,E706,It is recommended that the students have basic...,WebOodi,2018-10-30,21C00300 Henkilöstöjohtaminen,Kathrin Sele,Kathrin Sele,"Period II (2018-2019), Otaniemi campusPeriod I...",course,Lectures 30h presence (obligatory classroom pr...
3,3,,,True,,21C03000,The course is taught by a visiting lecturer an...,B.Sc. Management minor,1133021737,3-6,...,E706,,via WebOodi,2019-01-09,,The course is taught by a visiting lecturer. 2...,Mikko Martela,"2018-2019: III, Otaniemi campusNo teaching 201...",course,
4,4,,50% reflective learning diary50% final essay exam,True,,21C10000,"Must know: the concepts of ""concept and contex...",Aalto-course Management minor elective course,1121603277,6,...,E706,No specific prerequisites for attending the co...,Via Weboodi,2019-01-08,,Esko Aho Kirsti Iivonen,Esko Aho Kirsti Iivonen,Period III (2018-2019)Period III (2019-2020),course,Attending lectures 24h (not compulsory but hig...


In [42]:
# bla='creating stories and narratives'
# courses['name_low'].str.contains(r'^%s$'%bla).any()

In [43]:
def check_startdate(df):
    """Check if course starts in future"""
    df['startDate']=pd.to_datetime(df['startDate'])
    now=datetime.datetime.now()
    return df[df['startDate']>=now]

In [44]:
#create stemmer and tokenizer, to be used by tf-idf
#copied from https://github.com/senticr/SentiCR/blob/master/SentiCR/SentiCR.py
stemmer =SnowballStemmer("english")

def stem_tokens(tokens):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize_and_stem(text):
    tokens = word_tokenize(text)
    stems = stem_tokens(tokens)
    return stems

In [45]:
def top_tfidf(ind,tfidfvec,tfidf_matrix,n_tok=10):
    #tfidfvec,tfidf_matrix=define_tfidf(df,True,'english',False,None)
    features=tfidfvec.get_feature_names()
    course_tfidf=np.squeeze(tfidf_matrix[ind].toarray())
    sorted_tfidf=np.argsort(-course_tfidf) #minus cause want descending order
    return [(features[i],course_tfidf[i]) for i in sorted_tfidf[:n_tok]]

In [46]:
def get_mostsim(user_input,courses_ed,indices,sim):
    """Get the most similar courses for the input"""

    # Get index of course given title
    idx = indices[user_input]
    
    #Get similarity of course to all other courses
    # structure is list of (index, similarity)
    sim_row = list(enumerate(sim[idx]))

    #sort the courses by descending score
    sim_sorted = sorted(sim_row, key=lambda x: x[1], reverse=True)

    sim_sorted=[sim_sorted[i] for i in range(len(sim_sorted)) if courses_ed['name_low'].iloc[sim_sorted[i][0]] not in history_names]
    sim_indices = [i[0] for i in sim_sorted[1:]]
    sim_scores=[i[1] for i in sim_sorted[1:]] 

    return sim_indices,sim_scores

In [48]:
input_columns=['name','content']#['additionalInformation', 'assesmentMethods','content','courseStatus','credits','gradingScale','learningOutcomes','level','literature','organizationId','prerequisities','teacherInCharge','type','workload']#['content','name']
future=False
meas='cosine'
stopwords='english'
smooth=True
sublin=False
tokenize=tokenize_and_stem

In [ ]:
courses.columns

In [ ]:
courses['name_low']=courses['name'].str.lower()
courses['name_low']=courses['name_low'].str.strip()

In [ ]:
courses=courses.astype(str)

In [ ]:
courses['combined']=courses[input_columns].apply(lambda x: ' '.join(x), axis=1)

In [ ]:
#history tinka
history_names=['business and society','web software development','magnificent life','creating stories and narratives','digital workshop basics','machine learning: basic principles','algorithmic methods of data mining','machine learning: advanced probabilistic methods','information visualization','deep learning','law in digital society','bayesian data analysis','sustainable built environment','state of the world and development','finnish 1a','finnish 1b','finnish 2a']#['artificial intelligence','law in digital society','machine learning: advanced probabilistic methods','creating stories and narratives']

In [ ]:
#change from using names to using course codes?

In [ ]:
#history Jukka
#history_names=['theoretical computer science','computer graphics','information security','operating systems','linear algebra and differential equations','fourier analysis','matrix algebra','differential and integral calculus 1 (sci)','foundations of discrete mathematics','first course in probability and statistics','principles of algorithmic techniques','discrete models and search','machine learning: basic principles','declarative programming','seminar in computer science algorithms','special course in copmuter science gnetic algorithms','individual studies in computer science hands-on natural language processing study group','scalable cloud computing','mobile systems programming','advanced course in algorithms','distributed algorithms','computational complexity theory','programming parallel computers','algorithmic methods of data mining','artificial intelligence','machine learning: advanced probabilistic methods','information visualization','computer vision','speech recognition p','reinforcement learning']

In [ ]:
len(history_names)

In [ ]:
history_courses=courses[courses['name_low'].isin(history_names)]

In [ ]:
#history_courses.columns

In [ ]:
#history_courses

In [ ]:
#list(history_courses['name'])

In [ ]:
#courses from history that do not occur in dataset
#[i for i in history_names if i not in list(history_courses['name_low'])]

In [ ]:
#top tfidfs from history
top_tfidf(1212,tfidf_all,tfidf_matrix,10)

In [ ]:
histo_all=' '.join(history_courses['combined'])

In [ ]:
' '.join((histo_all,'hello i am cool'))

In [ ]:
courses_ed=courses.append({'combined':histo_all,'name_low':'history'},ignore_index=True)

In [ ]:
#define the tf-idf vectorizer
tfidf_all = TfidfVectorizer(stop_words=stopwords,smooth_idf=smooth,sublinear_tf=sublin,tokenizer=tokenize)
#get the tf-idf score for each word in each ontent description of each course
tfidf_matrix = tfidf_all.fit_transform(courses_ed['combined'])

In [ ]:
#Construct a reverse map of indices and courses
# we use this to map index to title and the other way around
indices = pd.Series(courses_ed.index, index=courses_ed['name_low'])

In [ ]:
sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
sim_indices,sim_scores=get_mostsim('history',courses_ed,indices,sim)

In [ ]:
urls=['https://oodi.aalto.fi/a/opintjakstied.jsp?Kieli=6&html=1&Tunniste='+courses_ed['code'].iloc[i] for i in sim_indices[:10]]

In [ ]:
courses_ed['name'].iloc[sim_indices[:10]],urls,sim_scores[:10]

In [ ]:
[top_tfidf(ind,tfidf_all,tfidf_matrix,1) for ind in sim_indices[:10]]

In [ ]:
def recom_freetext(user_input,courses,input_columns=['content','name'],future=False,meas='cosine',stopwords='english',smooth=True,sublin=False,tokenize=None):
    """
    Give recommendations based on the user input
    user_input: string user inputs
    courses: DataFrame with the courses
    input_columns: the columns of the courses dataframe we want to consider
    future: if True only recommend courses taking place in the future
    meas: indicates the similarity measure we use
    stopwords: indicates which stopwords should be removed
    smooth: whether to use smooth idf
    sublin: whether to use sublinear TF
    tokenize: whether to tokenize the input
    """
    user_input=user_input.lower()
    #create one column that contains all info we consider per course
    courses['combined']=courses[input_columns].apply(lambda x: ' '.join(x), axis=1)
    #some preprocessing on the name (title of courses)
    courses['name_low']=courses['name'].str.lower()
    courses['name_low']=courses['name_low'].str.strip()
    #check if user input is a course or not
    if not courses['name_low'].str.contains(r'^%s$'%user_input).any():
        print('Not a course')
        courses=courses.append({'combined':user_input,'name_low':user_input},ignore_index=True)
    else:
        print('A course')

    #create dataframe with courses that meet the requirements 
    if future:
        courses_req=check_startdate(courses)
    else:
        courses_req=courses

    #define the tf-idf vectorizer
    tfidf_all = TfidfVectorizer(stop_words=stopwords,smooth_idf=smooth,sublinear_tf=sublin,tokenizer=tokenize)
    #get the tf-idf score for each word in each ontent description of each course
    tfidf_matrix_all = tfidf_all.fit_transform(courses['combined'])
    
    #do the same but with only the courses that meet the requirements. However, use the vocabulary from the list of all courses
    tfidf_req=TfidfVectorizer(use_idf=True,vocabulary=tfidf_all.vocabulary_,stop_words=stopwords,smooth_idf=smooth,sublinear_tf=sublin,tokenizer=tokenize)
    tfidf_matrix_req=tfidf_req.fit_transform(courses_req['combined'])
    
    #Construct a reverse map of indices and courses
    # we use this to map index to title and the other way around
    indices = pd.Series(courses_req.index, index=courses_req['name_low'])
    
    #get the similarity scores
    if meas=='cosine':
        sim = linear_kernel(tfidf_matrix_req, tfidf_matrix_req)
    sim_indices,sim_scores=get_mostsim(user_input,indices,sim)
    urls=['https://oodi.aalto.fi/a/opintjakstied.jsp?Kieli=6&html=1&Tunniste='+courses_req['code'].iloc[i] for i in sim_indices[:10]]
    return courses_req['name'].iloc[sim_indices[:10]],urls,sim_scores[:10]

In [ ]:
def get_recom(user_input):
    df=pd.read_csv('../Data/filtered_courses.csv')
    df['content']=df['content'].fillna('')
    course_titles,urls,similarities=recom_freetext(user_input,df)
    return course_titles, urls,similarities

In [ ]:
get_recom('artificial intelligence')